# Connect-RSC
Connects to Rubrik Security Cloud(RSC)

Connecting to RSC via the SDK requires the use of a Service Account. A Service Account is a special account created inside RSC that is granted permissions it needs to run an automation task. Before continuing, please read the user guide section pertaining to Service Accounts and create one for your usage. That section can be found [here](https://docs.rubrik.com/en-us/saas/saas/service_account.html).

Below we will show examples of connecting to RSC using a Service Account in different ways. 

---

# Setup Environment
Once you have created the Service Account, you need to set up your environment.  

## Step 1: Install the Rubrik Security Cloud Powershell SDK. 
To install the Rubrik Security Cloud Powershell SDK. Run the below code to install the SDK on your machine. Installing the module only needs to be done once. If you have already installed the SDK, then you can proceed to Step 2.

In [ ]:
Install-Module -Name RubrikSecurityCloud

## Step 2: Import the Rubrik Security Cloud Powershell SDK
To import the Rubrik Security Cloud Powershell SDK. We can do that with the Import-Module command as shown below. 

In [1]:
Import-Module RubrikSecurityCloud

---

# Connect using a service account json file
When you created the Service Account in RSC, you were given the option to download the json file. You will have a file that has content that looks like the below. 

```json
{
	"client_id": "client|lasdfs1fZ9a2GYRBjrJqotseu",
	"client_secret": "CCzC_wY6PwVMOTdUZKktr8QL3OWlhasdlfhljasfIz314mErx",
	"name": "RSC Demo Service Account",
	"access_token_uri": "https://rubrik-demo.my.rubrik.com/api/client_token"
}
```

Treat this json file as a secure artifact. This file contains credential information to connect and interact with your RSC instance. This file should be placed into a secure location. 

To use the json file, run the code below. 

In [9]:
# ls ~/Documents/RSCDemoServiceAccount.json
Connect-RSC -ServiceAccountFile /Users/Chris.Lumnah/Documents/RSCDemoServiceAccount.json

Connect-Rsc does not support unencrypted JSON files for service accounts. Please use the 'Set-RscServiceAccountFile' cmdlet to create a valid credential file.
Connect-Rsc: 
Line |
   2 |  Connect-RSC -ServiceAccountFile /Users/Chris.Lumnah/Documents/RSCDemo …
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Data at the root level is invalid. Line 1, position 1.


# Connect using a service account file
## Create an encrypted format of the service account json file
`Set-RscServiceAccountFile` will generate an encrypted service account (SA) file, store it in a default location, and prompt you to delete the clear-text SA file. We recommend that you delete the clear-text SA file after you have generated the encrypted one to avoid any security issues (if you ever need a clear-text SA file again, they can easily be retrieved from the RSC UI).

You only need to do the step with `Set-RscServiceAccountFile` once.

In [10]:
Set-RscServiceAccountFile -InputFile /Users/Chris.Lumnah/Documents/RSCDemoServiceAccount.json


Creating credential store...

Reading RSC Service Account JSON File from /Users/Chris.Lumnah/Documents/RSCDemoServiceAccount.json

Saving credential file: /Users/Chris.Lumnah/.config/powershell/rubrik-powershell-sdk/rsc_service_account_default.xml

Rubrik strongly discourage keeping clear text service account files.


In [11]:
Connect-Rsc


ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20230607-45     v20230614-31



---

# Connect using an Environment Variable
An alternative is to store the location of the json file in an environment variable. Again, the file contains credential information and should be treated as a secure artifact. Store the file in a secure location. Connect-RSC looks for an environment variable called **RSC_SERVICE_ACCOUNT_FILE**

Since this notebook is showing Powershell examples, we will show how to create and use an environment variable in Powershell

In [14]:
# Create Environment Variable in pwsh. This can be done one time
$env:RSC_SERVICE_ACCOUNT_FILE = '/Users/Chris.Lumnah/Documents/RSCDemoServiceAccount.json'

In [15]:
# Connect to RSC
Connect-Rsc -FromEnv

Connect-Rsc does not support unencrypted JSON files for service accounts. Please use the 'Set-RscServiceAccountFile' cmdlet to create a valid credential file.
Connect-Rsc: 
Line |
   2 |  Connect-Rsc -FromEnv
     |  ~~~~~~~~~~~~~~~~~~~~
     | Data at the root level is invalid. Line 1, position 1.


---

# Connect using Service Account Values
If you do not have the json file, but you have all of the values from the json file, you can pass each value into the appropriate parameter. It is important to note that the Client Secret parameter is expecting a secure string. Your code will look something like...

```pwsh
$ClientID = "client|lasdfs1fZ9a2GYRBjrJqotseu"
$ClientSecret = ConvertTo-SecureString ("CCzC_wY6PwVMOTdUZKktr8QL3OWlhasdlfhljasfIz314mErx") -AsPlainText -Force
$AccessTokenURI = "https://rubrik-demo.my.rubrik.com/api/client_token"

Connect-RSC -Server $AccessTokenURI -ClientID $ClientID -ClientSecret $ClientSecret
```
Remember that the above values are security artifacts, so do not store these values in plain text in your scripts. They should be placed into a secure location. 


In the example below, we are taking things to the next level by having our values stored in a Password Vault. In this example, I am using 1Password. 


In [16]:
$ClientID = op read op://Rubrik/RSCDemoServiceAccount/username
$ClientSecret = ConvertTo-SecureString (op read op://Rubrik/RSCDemoServiceAccount/credential) -AsPlainText -Force
$AccessTokenURI = op read op://Rubrik/RSCDemoServiceAccount/hostname

#Connect to RSC
Connect-RSC -Server $AccessTokenURI -ClientID $ClientID -ClientSecret $ClientSecret

ServiceAccountDetails

ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20230607-45     v20230614-31



---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [17]:
Disconnect-Rsc

The Rubrik Security Cloud session has been terminated.
